In [296]:
from aocd import get_data
import numpy as np
from scipy.spatial import distance_matrix
from sklearn.linear_model import LinearRegression

day19 = get_data(day=19,year=2021)

In [297]:
test = '''--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14'''



In [324]:
scanners = [[j for j in i.split('\n')[1:] if j!=''] for i in test.split('--- scanner ')[1:]]
scanners = [np.array([[int(k) for k in j.split(',')] for j in i]) for i in scanners]
scanners[0]

array([[ 404, -588, -901],
       [ 528, -643,  409],
       [-838,  591,  734],
       [ 390, -675, -793],
       [-537, -823, -458],
       [-485, -357,  347],
       [-345, -311,  381],
       [-661, -816, -575],
       [-876,  649,  763],
       [-618, -824, -621],
       [ 553,  345, -567],
       [ 474,  580,  667],
       [-447, -329,  318],
       [-584,  868, -557],
       [ 544, -627, -890],
       [ 564,  392, -477],
       [ 455,  729,  728],
       [-892,  524,  684],
       [-689,  845, -530],
       [ 423, -701,  434],
       [   7,  -33,  -71],
       [ 630,  319, -379],
       [ 443,  580,  662],
       [-789,  900, -551],
       [ 459, -707,  401]])

In [325]:
def scanner_matches(sc0,sc1):
    npairs = len(set(distance_matrix(sc0,sc0).flatten())&set(distance_matrix(sc1,sc1).flatten())) - 1
    return np.sqrt(2*npairs + 1/4) - 1/2 + 1

scanner_matches(beacons,scanners[3])

2.5615528128088303

In [326]:
def get_matches(sc0,sc1):
#Distance matrices
    if scanner_matches(sc0,sc1) < 12:
        return
    dist0 = distance_matrix(sc0,sc0)
    dist1 = distance_matrix(sc1,sc1)

    #Matching values
    matching = set(dist0.flatten())&set(dist1.flatten())

    #Indices of matching values
    ind0 = np.where(np.array([i in matching for i in dist0.flatten()]).reshape(dist0.shape).sum(axis=1)>=12)
    ind1 = np.where(np.array([i in matching for i in dist1.flatten()]).reshape(dist1.shape).sum(axis=1)>=12)

    match0 = sc0[ind0]
    match1 = sc1[ind1]

    dist0 = distance_matrix(match0,match0)
    match0 = match0[np.argsort(dist0.mean(axis=1))]
    dist1 = distance_matrix(match1,match1)
    match1 = match1[np.argsort(dist1.mean(axis=1))]
    
    return match0,match1

get_matches(scanners[0],scanners[1])

(array([[-537, -823, -458],
        [-447, -329,  318],
        [-345, -311,  381],
        [-485, -357,  347],
        [ 459, -707,  401],
        [ 423, -701,  434],
        [ 390, -675, -793],
        [-618, -824, -621],
        [-661, -816, -575],
        [ 528, -643,  409],
        [ 404, -588, -901],
        [ 544, -627, -890]]),
 array([[ 605,  423,  415],
        [ 515,  917, -361],
        [ 413,  935, -424],
        [ 553,  889, -390],
        [-391,  539, -444],
        [-355,  545, -477],
        [-322,  571,  750],
        [ 686,  422,  578],
        [ 729,  430,  532],
        [-460,  603, -452],
        [-336,  658,  858],
        [-476,  619,  847]]))

In [343]:
sc0,sc1 = scanners[0],scanners[1]
def get_transformed(sc0,sc1):
    match0,match1 = get_matches(sc0,sc1)
    reg = LinearRegression().fit(match1,match0)
    return np.rint(reg.predict(sc1))

get_transformed(sc0,sc1)

array([[ -618.,  -824.,  -621.],
       [ -537.,  -823.,  -458.],
       [ -447.,  -329.,   318.],
       [  404.,  -588.,  -901.],
       [  -27., -1108.,   -65.],
       [  544.,  -627.,  -890.],
       [  408., -1815.,   803.],
       [ -499., -1607.,  -770.],
       [  528.,  -643.,   409.],
       [ -601., -1648.,  -643.],
       [ -661.,  -816.,  -575.],
       [  568., -2007.,  -577.],
       [  390.,  -675.,  -793.],
       [  534., -1912.,   768.],
       [  497., -1838.,  -617.],
       [  423.,  -701.,   434.],
       [ -635., -1737.,   486.],
       [  396., -1931.,  -563.],
       [ -345.,  -311.,   381.],
       [  459.,  -707.,   401.],
       [ -518., -1681.,  -600.],
       [  432., -2009.,   850.],
       [ -739., -1745.,   668.],
       [ -687., -1600.,   576.],
       [ -485.,  -357.,   347.]])

In [345]:
beacons = scanners[0]
done = np.repeat(False,len(scanners))
done[0] = True
while not np.all(done):
    for i in range(len(scanners)):
        if not done[i]:
            sc = scanners[i]
            num = scanner_matches(beacons,sc)
            if num >= 12:
                new = get_transformed(beacons,sc)
                beacons = np.unique(np.append(beacons,new,axis=0),axis=0)
                done[i] = True

In [346]:
len(beacons)

79

Okay, the example works! Now let's try the problem.

In [348]:
scanners = [[j for j in i.split('\n')[1:] if j!=''] for i in day19.split('--- scanner ')[1:]]
scanners = [np.array([[int(k) for k in j.split(',')] for j in i]) for i in scanners]

beacons = scanners[0]
done = np.repeat(False,len(scanners))
done[0] = True
while not np.all(done):
    for i in range(len(scanners)):
        if not done[i]:
            sc = scanners[i]
            num = scanner_matches(beacons,sc)
            if num >= 12:
                new = get_transformed(beacons,sc)
                beacons = np.unique(np.append(beacons,new,axis=0),axis=0)
                done[i] = True

len(beacons)

378

In [351]:
def get_scanner(sc0,sc1):
    match0,match1 = get_matches(sc0,sc1)
    reg = LinearRegression().fit(match1,match0)
    return reg.intercept_

In [352]:
scanners = [[j for j in i.split('\n')[1:] if j!=''] for i in day19.split('--- scanner ')[1:]]
scanners = [np.array([[int(k) for k in j.split(',')] for j in i]) for i in scanners]

scannerloc = []

beacons = scanners[0]
done = np.repeat(False,len(scanners))
done[0] = True
while not np.all(done):
    for i in range(len(scanners)):
        if not done[i]:
            sc = scanners[i]
            num = scanner_matches(beacons,sc)
            if num >= 12:
                new = get_transformed(beacons,sc)
                beacons = np.unique(np.append(beacons,new,axis=0),axis=0)
                scannerloc.append(get_scanner(beacons,sc))
                done[i] = True

In [353]:
scannerloc

[array([1117.,  110.,    2.]),
 array([-118.,   62., 1184.]),
 array([1157.,  125., 1081.]),
 array([2428.,   -8., 1258.]),
 array([ 1213., -1163.,   -83.]),
 array([ 1106., -2264.,    11.]),
 array([ -119., -2390.,   -58.]),
 array([ 1146., -1076.,  1131.]),
 array([2358.,   35.,   55.]),
 array([ 1066., -1172.,  2432.]),
 array([-1.223e+03,  5.500e+01,  1.000e+00]),
 array([1195.,   48., 2365.]),
 array([2321., 1180.,  -92.]),
 array([3594.,  -16.,    5.]),
 array([2259., 2358., -125.]),
 array([-1206., -2265.,    60.]),
 array([1205., 1180., 2285.]),
 array([2292., 3715.,   68.]),
 array([3525., 2487., -110.]),
 array([3499., 3664.,   41.]),
 array([3597., 1300.,   46.]),
 array([-1195., -3575.,   -30.]),
 array([4638., 2451., -106.]),
 array([ 3504.,  1269., -1287.]),
 array([3532., 1263., 1097.]),
 array([ 3500.,  3631., -1219.]),
 array([ 3584., -1097.,    28.]),
 array([ 3552.,  2494., -1276.]),
 array([4662., 1149.,    6.]),
 array([ 3549.,  2476., -2383.])]

In [356]:
np.max(distance_matrix(scannerloc,scannerloc,p=1))

13148.0

Got it!